# Process SDII Data (Basic)

**This notebook demonstrates how to read and perform basic exploratory analysis on archived and streaming SDII data using the Python SDK. SDII stands for Sensor Data Ingestion Interface. [Learn more about the SDII format specification](https://developer.here.com/documentation/sdii-data-spec/topics/introduction.html "SDII Documentation")**


### Dependencies
* Catalogs: [SDII Sample Berlin](https://platform.here.com/data/hrn:here:data::olp-here:olp-sdii-sample-berlin-2)  
* Layers: [Sample Versioned Layer](https://platform.here.com/data/hrn:here:data::olp-here:olp-sdii-sample-berlin-2/sample-versioned-layer)
* Languages: Python


### Test data sample description
The data sample contains simulated vehicle travel paths and corresponding simulated traffic sign observations. Although the schema represents the actual SDII data structure not all fields have been populated. The simulated signs are for demonstration purposes only and do not represent actual sign placements. 

### Workflow
- Import packages
- Import Python SDK Libraries
- Specify data catalog and layer
- Specify a bounding box and get tiles
- Read SDII data from a versioned layer in HERE deserialize using the Python SDK utilities
- Extract information of interest to _pandas_ dataframes for manipulation and visualization
- Perform simple exploratory analysis
- Read SDII data from a streaming layer and print some trace information

## Import Python Packages

In [1]:
!pip install matplotlib 

                                              0.0/7.6 MB ? eta -:--:--
                                              0.0/7.6 MB 960.0 kB/s eta 0:00:08
     -                                        0.3/7.6 MB 3.8 MB/s eta 0:00:02
     ---                                      0.7/7.6 MB 5.6 MB/s eta 0:00:02
     -----                                    1.1/7.6 MB 6.5 MB/s eta 0:00:02
     --------                                 1.5/7.6 MB 7.0 MB/s eta 0:00:01
     ----------                               1.9/7.6 MB 7.2 MB/s eta 0:00:01
     ------------                             2.3/7.6 MB 7.4 MB/s eta 0:00:01
     --------------                           2.8/7.6 MB 7.6 MB/s eta 0:00:01
     ----------------                         3.1/7.6 MB 7.7 MB/s eta 0:00:01
     ------------------                       3.5/7.6 MB 7.9 MB/s eta 0:00:01
     --------------------                     3.9/7.6 MB 7.8 MB/s eta 0:00:01
     ----------------------                   4.3/7.6 MB 8.0 MB/s eta

In [2]:
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import operator

This configuration is being used for bar and chart.

In [5]:
plt.rcParams['figure.figsize'] = (20, 6)
plt.rcParams['axes.linewidth'] = 3
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 18

## Import the Python SDK libraries

In [15]:
from here.platform import Platform



ModuleNotFoundError: No module named 'here.inspector'

# Initialize the platform object

In [18]:
platform = Platform()
platform.tpeg2geojson('data/traffic.tpeg', 'data/traffic.geojson')

AttributeError: 'Platform' object has no attribute 'tpeg2geojson'

## Specify data catalog and layer

In [14]:
CATALOG_HRN = 'hrn:here:data::olp-here:olp-sdii-sample-berlin-2'
LAYER_ID = 'sample-versioned-layer'

catalog = platform.get_catalog(CATALOG_HRN)
layer = catalog.get_layer(LAYER_ID)

AuthenticationException: An error occurred during authentication or authorization with HERE
                    platform: Status 401 -
                    Reason Unauthorized
 Response: {"errorId":"ERROR-ab49d345-34b4-42d9-9163-7cdc86ebc939","httpStatus":401,"errorCode":401300,"message":"Invalid client credentials.","error":"invalid_client","error_description":"errorCode: '401300'. Invalid client credentials."}

## Specify a bounding box for the data and get the corresponding tiles

In [ ]:
tile_ids = list(ht.between_points(Point(13.3, 52.5), Point(13.5, 52.6), level=14, fully_contained=False))
tile_ids[:10]

## Read and parse the SDII data

In [ ]:
blobs = layer.read_partitions(tile_ids)

# We select only the decoded messages, ignoring partition information
parsed_blobs = [ blob for _, blob in blobs ]
parsed_blobs[:10]

## Calulate  number of occurance of RoadSignData

In [ ]:
def get_freq(arr):
    elements_count = {}
    for element in arr:
        if element in elements_count:
            elements_count[element] += 1
        else:
            if element is not None:
                elements_count[element] = 1
    return elements_count.items()

### Assign Data to RoadSignFields

In [ ]:
roadSignType_arr2= []
roadSignPermanency_arr= []
roadSignValue_arr= []
messageId_arr= []
messageId_count_arr= []
messageId_count_dict= {}
messageId_lng_lat_dict= {}

for blob in parsed_blobs:
    for item in getattr(blob, 'messages'):
        messageId_arr.append(item.messageId)
        messageId_count_arr = list(item.message.path.positionEstimate)
        messageId_count_dict[item.messageId] = len(messageId_count_arr)
        messageId_lng_lat_dict[item.messageId] = messageId_count_arr
        for item2 in item.message.pathEvents.signRecognition:
            roadSignType_arr2.append(item2.roadSignType)
            roadSignPermanency_arr.append(item2.roadSignPermanency)
            if item2.roadSignValue is not '':
                roadSignValue_arr.append(item2.roadSignValue)

### Fetch RoadSignFields values and their occurance

In [ ]:
roadSignType_count_dict= dict(get_freq(roadSignType_arr2))
roadSignType_count_dict = dict(sorted(roadSignType_count_dict.items()))
roadSignValue_count_dict = dict(get_freq(roadSignValue_arr))
messageId_arr= dict(get_freq(messageId_arr))

### Number of drives per archiving tile - drives are associated with tiles based on the first GPS point in the trace

In [ ]:
def getMsgCountGroupByTileId(blobs):
    data = dict()
    for blob in blobs:
        data[blob.tileId] = len(blob.messages)
    return data

In [ ]:
parsed_data = getMsgCountGroupByTileId(parsed_blobs)
df = parsed_data
plt.bar(range(len(df)), list(df.values()), align='center')
plt.xticks(range(len(df)), list(df.keys()))
plt.xlabel('Tile ID')
plt.ylabel('Number of messages')
plt.grid(True)

plt.show()

## Traffic sign information

### Statistics for different sign types in the sample

In [ ]:
cmap = plt.get_cmap("tab20c")

labels_signs = [str(h) for h in roadSignType_count_dict.keys()]

pie = plt.pie(roadSignType_count_dict.values(), shadow=False, startangle=140, pctdistance=0.85, colors=cmap.colors)

plt.legend(pie[0],labels_signs, bbox_to_anchor=(1,0.5), loc="center right", fontsize=12,
           bbox_transform=plt.gcf().transFigure)

plt.tight_layout()

### Number of signs by permanency

In [ ]:
roadSignPermanency = set(roadSignPermanency_arr)

fig, ax =plt.subplots(1,1)
data=[[list(roadSignPermanency)[0],len(roadSignPermanency_arr)]]
column_labels=["roadSignPermanency", "count"]
ax.axis('tight')
ax.axis('off')
ax.table(cellText=data,colLabels=column_labels,loc="center")

plt.show()

### Number of observation of speed limit signs by value

In [ ]:
plt.bar(range(len(roadSignValue_count_dict)), list(roadSignValue_count_dict.values()), align='center')
plt.xticks(range(len(roadSignValue_count_dict)), list(roadSignValue_count_dict.keys()))
plt.xlabel('Speed sign value')
plt.ylabel('Number of signs')
plt.grid(True)

plt.show()

### Number of GPS points recorded in each drive (sorted from longest to shortest trace)

In [ ]:
msgs_by_count = messageId_count_dict
max_obs = max(messageId_count_dict.values())
bins=list(range(1,max_obs+2, 5))

In [ ]:
# Implementation of matplotlib function
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(10**7)
mu = 1
sigma = 21
x = messageId_count_dict.values()
num_bins = bins

n, num_bins, patches = plt.hist(x, bins)

y = ((1 / (np.sqrt(2 * np.pi) * sigma)) *
    np.exp(-1 * (1 / sigma * (num_bins - mu))**2))

plt.plot(bins, y)

plt.title('Number of GPS points per message')
plt.xlabel('Number of GPS points')
plt.ylabel('Number of messages')
plt.grid(True)

plt.show()

### Get the longest drive (based on number of GPS points)

In [ ]:
msgs_by_count_sorted = dict( sorted(msgs_by_count.items(), key=operator.itemgetter(1),reverse=True))
frq_msg = list(msgs_by_count_sorted.keys())[0]
lat_lon_list = list(messageId_lng_lat_dict[frq_msg])

points = [ Point(item.longitude_deg, item.latitude_deg) for item in lat_lon_list ]

inspect(points, style=Color.BLUE)

# Loading SDII data from streaming layers

In [ ]:
LAYER_ID = 'sample-streaming-layer'

layer = catalog.get_layer(LAYER_ID)
subscription = layer.subscribe()

# get data formatted similar to a partition object
parsed_msgs = layer.read_stream(subscription=subscription)

parsed_data_list = [ msg.path.positionEstimate for _, msg in parsed_msgs ]

result_data = []
for i in range(len(parsed_data_list[0])):
    result_data.append([])
    result_data[i].append(parsed_data_list[0][i].timeStampUTC_ms)
    result_data[i].append(parsed_data_list[0][i].latitude_deg)
    result_data[i].append(parsed_data_list[0][i].longitude_deg)
    result_data[i].append(parsed_data_list[0][i].heading_deg)
    result_data[i].append(parsed_data_list[0][i].speed_mps)
    if i == 4:
        break

fig, ax = plt.subplots(1,1)
column_labels=["timeStampUTC_ms", "latitude_deg", "longitude_deg", "heading_deg", "speed_mps"]
ax.axis('tight')
ax.axis('off')
ax.table(cellText=result_data,colLabels=column_labels,loc="center")

plt.show()

### Cleaning up

When done, unsubscribe from the stream.

In [ ]:
subscription.unsubscribe()

<span style="float:left; margin-top:3px;"><img src="https://www.here.com/themes/custom/here_base_theme_v2/logo.svg" alt="HERE Logo" height="60" width="60"></span><span style="float:right; width:90%;"><sub><b>Copyright (c) 2020-2021 HERE Global B.V. and its affiliate(s). All rights reserved.</b>
This software, including documentation, is protected by copyright controlled by HERE. All rights are reserved. Copying, including reproducing, storing, adapting or translating, any or all of this material requires the prior written consent of HERE. This material also contains confidential information which may not be disclosed to others without the prior written consent of HERE.</sub></span>